Request

- User ID
- User searching country
- expecting shops count as result


Response

- listed shops id arrays

User   

userID  
Q1={'Beauty & Wellness', 'Events & Spaces'}
Q2={'Salons', 'Spa'}
Q3={'Hair cut', 'Nail Care'}
Q4=18-24
Q5=male
Q6=2000-5000
Q7=colombo


User Behavior  

ShopID  
userID  
Booking_count  
Average_Booking_cost  


Shop  

Shop ID  
ShopName  
country 
Location_name  
Location_long  
Location_lat  
Description 
Shop_categories (beauty & wellness, ...) 
Shop_categories (spa, saloon)   
Available_package_types (hair care, nail care)
Total_Paid_Ads_Count_Current_Month  
Total_Reviews_Count_Current_Month  
Total_Bookings_Count_Current_Month  
  

Shop Behaviors  

Shop_ID  
Total_Paid_Ads_Count_Current_Month  
Total_Reviews_Count_Current_Month  
Total_Bookings_Count_Current_Month

In [12]:
import pandas as pd


from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.neighbors import NearestNeighbors

In [25]:
user_data = pd.read_excel('Users_Data.xlsx')
user_data.head()

,UserID,Q1,Q2,Q3,Q4,Q6,Q7,Q8,Q10
0,1,{'Education & Experts'},{'Therapy'},25-34,Male,"{'Skin Care', 'Nail Care', 'Hair cut'}","LKR 5,000 - LKR 10,000",Galle,{'Other'}
1,2,{'In-Demand Ser'},{'Therapy'},45-54,Male,"{'Nail Care', 'Hair cut'}","LKR 2,000 - LKR 5,000",Negombo,{'Keratin or protein treatments'}
2,3,{'Fashion & Style'},{'Cosmetic Services'},55+,Female,"{'Nail Care', 'Hair cut'}","LKR 5,000 - LKR 10,000",Jaffna,{'Scalp treatments and massages'}
3,4,{'Fashion & Style'},{'Cosmetic Services'},35-44,Female,"{'Skin Care', 'Nail Care', 'Hair cut'}","LKR 10,000 - LKR 15,000",Nugegoda,{'Other'}
4,5,{'Education & Experts'},{'Therapy'},25-34,Male,{'Skin Care'},"LKR 20,000+",Kandy,"{'Hair colouring (e.g., balayage, highlights)'..."


In [ ]:
user_behavior = pd.read_excel('User_Behavior.xlsx')
user_behavior.head()

In [ ]:
shop_data = pd.read_excel('Shop_Data.xlsx')
shop_data.head()

### Handle Array-like Columns:

In [26]:
mlb = MultiLabelBinarizer()

Q1_binarized = pd.DataFrame(mlb.fit_transform(user_data['Q1']), columns=mlb.classes_, index=user_data.index)
Q2_binarized = pd.DataFrame(mlb.fit_transform(user_data['Q2']), columns=mlb.classes_, index=user_data.index)
Q6_binarized = pd.DataFrame(mlb.fit_transform(user_data['Q6']), columns=mlb.classes_, index=user_data.index)
Q10_binarized = pd.DataFrame(mlb.fit_transform(user_data['Q10']), columns=mlb.classes_, index=user_data.index)
# Add binarized data to user_data
user_data = pd.concat([user_data, Q1_binarized, Q2_binarized,Q6_binarized,Q10_binarized], axis=1)


In [27]:
user_data['Q3'] = user_data['Q3'].astype('category').cat.codes
user_data['Q4'] = user_data['Q4'].astype('category').cat.codes
user_data['Q7'] = user_data['Q7'].astype('category').cat.codes
user_data['Q8'] = user_data['Q8'].astype('category').cat.codes

In [28]:
user_data = user_data.drop(['Q1', 'Q2', 'Q6', 'Q10'], axis=1)

In [29]:
user_data.head()

,UserID,Q3,Q4,Q7,Q8,,&,',",",-,...,o,p,r,s,t,u,w,y,{,}
0,1,1,1,4,2,1,1,1,0,0,...,0,0,1,0,1,0,0,0,1,1
1,2,3,1,2,6,1,0,1,0,1,...,1,1,1,1,1,0,0,0,1,1
2,3,4,0,4,3,1,1,1,0,0,...,0,1,1,1,1,0,0,0,1,1
3,4,2,0,0,7,1,1,1,0,0,...,0,0,1,0,1,0,0,0,1,1
4,5,1,1,3,4,1,1,1,0,0,...,1,0,1,1,1,1,1,1,1,1


In [31]:
user_data.shape


(100, 119)

### Define feature columns

In [32]:
X = user_data.drop(['UserID'], axis=1)  # Features
user_ids = user_data['UserID']  # User IDs


In [33]:
knn_model = NearestNeighbors(n_neighbors=10, metric='cosine')
knn_model.fit(X)


NearestNeighbors(metric='cosine', n_neighbors=10)

In [34]:
new_user_data = {
    'Q1': {'Beauty & Wellness', 'Travel & Stay'},
    'Q2': {'Spa', 'Salon'},
    'Q3': '18-24',
    'Q4': 'Female',
    'Q6' : {'Skin Care', 'Nail Care', 'Hair cut'},
    'Q7': 'LKR 10,000 - LKR 15,000',
    'Q8': 'Colombo',
    'Q10': {'Classic manicures and pedicures'}
}

# Convert to the same format as training data
new_user = pd.DataFrame([new_user_data])
new_user['Q3'] = new_user['Q3'].astype('category').cat.codes
new_user['Q4'] = new_user['Q4'].astype('category').cat.codes
new_user['Q7'] = new_user['Q7'].astype('category').cat.codes
new_user['Q8'] = new_user['Q8'].astype('category').cat.codes

Q1_binarized = pd.DataFrame(mlb.transform(new_user['Q1']), columns=mlb.classes_)
Q2_binarized = pd.DataFrame(mlb.transform(new_user['Q2']), columns=mlb.classes_)
Q6_binarized = pd.DataFrame(mlb.transform(new_user['Q6']), columns=mlb.classes_)
Q10_binarized = pd.DataFrame(mlb.transform(new_user['Q10']), columns=mlb.classes_)

# Drop original columns after encoding
new_user = new_user.drop(['Q1', 'Q2', 'Q6', 'Q10'], axis=1)


new_user = pd.concat([new_user, Q1_binarized, Q2_binarized,Q6_binarized,Q10_binarized], axis=1)



c:\Users\DELL\.conda\envs\condaenv\Lib\site-packages\sklearn\preprocessing\_label.py:900: UserWarning: unknown class(es) ['Beauty & Wellness', 'Travel & Stay'] will be ignored
  warnings.warn(
c:\Users\DELL\.conda\envs\condaenv\Lib\site-packages\sklearn\preprocessing\_label.py:900: UserWarning: unknown class(es) ['Salon', 'Spa'] will be ignored
  warnings.warn(
c:\Users\DELL\.conda\envs\condaenv\Lib\site-packages\sklearn\preprocessing\_label.py:900: UserWarning: unknown class(es) ['Hair cut', 'Nail Care', 'Skin Care'] will be ignored
  warnings.warn(
c:\Users\DELL\.conda\envs\condaenv\Lib\site-packages\sklearn\preprocessing\_label.py:900: UserWarning: unknown class(es) ['Classic manicures and pedicures'] will be ignored
  warnings.warn(


In [35]:
distances, indices = knn_model.kneighbors(new_user)

# Get the most similar users
similar_users = user_ids.iloc[indices[0]].values
print("Similar Users:", similar_users)


ValueError: The feature names should match those that were passed during fit.
Feature names seen at fit time, yet now missing:
- &
- -
- D
- E
- F
- ...
